# Around the World with CIA Factbook (2015 Edition)

Let's take a quick virtual tour around the world with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a concise collection of information about all of the countries on Earth.

We'll explore the countries that:
- Have most people, least people, and even no people
- Added most people into the world on 2016
- Are most packed with people (hint: not China!)
- Have more waters than land

![World atlas global vegetation](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6a/Global_Vegetation_.jpg/640px-Global_Vegetation_.jpg)

## Things to do and things to know

First, let's load the database and see what's inside.

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [3]:
%%sql
SELECT *
    FROM sqlite_master
    WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [4]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Some descriptions of the columns in the file that I will use for analysis.

- `name` — the name of the country.
- `area` — the country's total area (both land and water).
- `area_land` — the country's land area in square kilometers.
- `area_water` — the country's waterarea in square kilometers.
- `population` — the country's population.
- `population_growth` — the country's population growth as a percentage.
- `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

There's also a `code` column, which I thought was ISO 3166 two-digit code for country, but actually it's not. But fret not, CIA provides a handy [cross-reference list](http://https://www.cia.gov/library/publications/the-world-factbook/appendix/appendix-d.html) of their country code to ISO 3166 code just in case you need it.

## Summary statistics (with cute pics)
Let's have a quick look on the world's population.

In [6]:
%%sql
SELECT MIN(population) AS 'Min population',
    MAX(population) AS 'Max population',
    MIN(population_growth) AS 'Min population growth',
    MAX(population_growth) AS 'Max population growth'
    FROM facts;

Done.


Min population,Max population,Min population growth,Max population growth
0,7256490011,0.0,4.02


Let's find out more about the funny numbers on `Min population` and `Max population`.

In [61]:
%%sql
SELECT name AS country, population
    FROM facts
    WHERE population == (SELECT MIN(population)
                                   FROM facts
                        );

Done.


country,population
Antarctica,0


This is what the [Factbook says](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html) about Antarctica:

>no indigenous inhabitants, but there are both permanent and summer-only staffed research stations"

So, population 0 makes sense.

No worries, it's not all that bad, we still have some friendly penguins there! Just like little Fred here.
<img src = "https://upload.wikimedia.org/wikipedia/commons/0/08/South_Shetland-2016-Deception_Island%E2%80%93Chinstrap_penguin_%28Pygoscelis_antarctica%29_04.jpg" width="300" height="400">

To be honest, Antarctica is listed at the top of my bucket list. You can go there through Argentina or New Zealand in [a fancy cruise](https://www.nationalgeographic.com/expeditions/destinations/polar/ocean/antarctica-cruise/) to take cool pics of penguins, yachting, camping, hiking and cross country skiing in South Pole!
<img src="https://upload.wikimedia.org/wikipedia/commons/1/17/National_Geographic_Explorer_in_fast_ice%2C_Antarctica_-_edit_1.jpg" width="600" height="400">

Other than that, I also have an unhealthy obsession to see polar bears in North Pole. Let's see what the Factbook says about Arctic (which is called 'Arctic Ocean' in this database).

In [62]:
%%sql
SELECT name AS Country, population
    FROM facts
    WHERE name == 'Arctic';

Done.


Country,population


Eh, why is there no Arctic if there is an Antarctica? I demand my cute polar bears!
<img src="https://upload.wikimedia.org/wikipedia/commons/2/20/Ursus_maritimus_us_fish.jpg" width="600" height="400">

Ah OK, so Arctic is called ['Arctic Ocean'](https://www.cia.gov/library/publications/the-world-factbook/geos/xq.html) based on the Factbook.

In [63]:
%%sql
SELECT name AS Country, population
    FROM facts
    WHERE name == 'Arctic Ocean';

Done.


Country,population
Arctic Ocean,None


OK, why `population` is 'None' and not 0?

Out of curiosity, let's just check is there any other country with `population` 'None' (which is NULL) instead of 0.

In [64]:
%%sql
SELECT name AS Country, population
    FROM facts
    WHERE population IS NULL
    ORDER BY name;

Done.


Country,population
Arctic Ocean,None
Ashmore and Cartier Islands,None
Atlantic Ocean,None
Bouvet Island,None
British Indian Ocean Territory,None
Clipperton Island,None
Coral Sea Islands,None
French Southern and Antarctic Lands,None
Heard Island and McDonald Islands,None
Indian Ocean,None


Oh so we have several NULLs here for countries that are islands and oceans, but why is there an inconsistency between having the NULL value and 0 value?

Anyway, let's end our fascinating polar expeditions right here and let's see the top country that has the lowest, non-0, and non-NULL population.

In [150]:
%%sql
SELECT name AS Country, MIN(population) AS 'Population'
    FROM facts
    WHERE population != 0
    AND population != 'NULL';

Done.


Country,Population
Pitcairn Islands,48


Here's a pic of Adamstown, the only settlement on the Pitcairn island (no one lives on the other three islands (or atolls) named Ducie, Henderson and Oeno).
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/75/Adamstown1.jpg/1024px-Adamstown1.jpg" width="600" height="400">
Looks like a beautiful place!

Now let's talk about the country with maximum population.

In [65]:
%%sql
SELECT name AS Country, population
    FROM facts
    WHERE population == (SELECT MAX(population)
                                   FROM facts
                        );

Done.


Country,population
World,7256490011


Well, 'World' is not a country, so let's exclude it and see what we got. I'm guessing it's China that has most people.

In [66]:
%%sql
SELECT name AS Country, population
    FROM facts
    WHERE population == (SELECT MAX(population)
                                   FROM facts
                         WHERE name != 'World'
                        );

Done.


Country,population
China,1367485388


Yes, it's China, so here's a cute panda pic, featuring [Yuan Zai](https://en.wikipedia.org/wiki/Yuan_Zai_(giant_panda) the giant panda!
![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ac/Yuan_Zai_Taipei_Zoo.jpg/640px-Yuan_Zai_Taipei_Zoo.jpg)

OK since we're now in Asia, let's stop by my country, featuring the famous [Raja Ampat](https://www.indonesia.travel/id/en/destinations/maluku-papua/raja-ampat) in the eastern part of Indonesia.
<img src="https://upload.wikimedia.org/wikipedia/commons/b/b5/Beautiful_Raja_Ampat.jpg" width="600" height="400">

In [19]:
%%sql
SELECT *
    FROM facts
    WHERE name == 'Indonesia';

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
78,id,Indonesia,1904569,1811569,93000,255993674,0.92,16.72,6.37,1.16


But I'm pretty sure my country's `area_water` is [larger](https://en.wikipedia.org/wiki/Geography_of_Indonesia) than `area_land`... hmmm. Water territory is quite a sensitive national issue here.
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/ID_TW_EEZ.svg/1920px-ID_TW_EEZ.svg.png" width="600" height="400">

And the birth rate is, like, almost 3x compared to the death rate. Should our government reinforce the mandatory birth control like several decades ago?

## Recalculating the summary statistics
Now that we found the value that might skew our data, let's do some recalculation.

In [20]:
%%sql
SELECT MIN(population) AS 'Min population',
    MAX(population) AS 'Max population',
    MIN(population_growth) AS 'Min population growth',
    MAX(population_growth) AS 'Max population growth'
    FROM facts
    WHERE name != 'World';

Done.


Min population,Max population,Min population growth,Max population growth
0,1367485388,0.0,4.02


Since the data looks much better now, let's calculate the average.

In [152]:
%%sql
SELECT ROUND(AVG(population), 0) AS 'Average population',
    ROUND(AVG(area), 2) AS 'Average area'
    FROM facts
    WHERE name != 'World';

Done.


Average population,Average area
32242667.0,555093.55


Let's take a quick check on how my country's population and area compared to the world's averages.

In [67]:
%%sql
SELECT name AS Country, population
    FROM facts
    WHERE population = (SELECT AVG(population)
                       FROM facts)
    OR name == 'Indonesia';

Done.


Country,population
Indonesia,255993674


In [153]:
world_average_population = 32242667
Indonesia_population = 255993674

world_average_area = 555093.55
Indonesia_area = 1904569
        
if Indonesia_area > world_average_area:
    print("Indonesia's area is higher than the world's average by " + "{:.2f}".format((Indonesia_area - world_average_area)/world_average_area) + "%")
else:
        print("Indonesia's area is lower than the world's average by " + "{:.2f}".format((world_average_area - Indonesia_area)/world_average_area) + "%")

if Indonesia_population > world_average_population:
    print("Indonesia's population is higher than the world's average by " + "{:.2f}".format((Indonesia_population - world_average_population)/world_average_population) + "%")
else:
        print("Indonesia's population is lower than the world's average by " + "{:.2f}".format((world_average_population - Indonesia_population)/world_average_population) + "%")

Indonesia's area is higher than the world's average by 2.43%
Indonesia's population is higher than the world's average by 6.94%


Next, let's see all countries with:
- above average `population`
- below average `area`

In [68]:
%%sql
SELECT name AS Country, population, area
    FROM facts
    WHERE population > (SELECT AVG(population)
                        FROM facts
                        WHERE name != 'World'
                       )
    AND area < (SELECT AVG(area)
               FROM facts
               WHERE name != 'World'
               );

Done.


Country,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


## The Top 10s

### Top 10 countries with the highest growth rate

In [96]:
%%sql
SELECT name AS Country, population_growth AS 'Growth rate'
    FROM facts
    ORDER BY population_growth DESC
    LIMIT 10;

Done.


Country,Growth rate
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24
Qatar,3.07
Burkina Faso,3.03
Mali,2.98
Cook Islands,2.95
Iraq,2.93


Almost all of these countries are located in the African continent.

### Top 10 countries with lowest growth rate

In [79]:
%%sql
SELECT name AS Country, population_growth AS 'Growth rate'
    FROM facts
    WHERE population_growth != 'NULL'
    ORDER BY population_growth
    LIMIT 10;

Done.


Country,Growth rate
Holy See (Vatican City),0.0
Cocos (Keeling) Islands,0.0
Greenland,0.0
Pitcairn Islands,0.0
Greece,0.01
Norfolk Island,0.01
Tokelau,0.01
Falkland Islands (Islas Malvinas),0.01
Guyana,0.02
Slovakia,0.02


### Top 10 countries with the highest water-to-land ratio

In [129]:
%%sql
SELECT name AS Country, area_water, area_land,
    ROUND(CAST(area_water AS Float)/CAST(area_land AS Float), 1) AS water_to_land_ratio
    FROM facts
    ORDER BY water_to_land_ratio DESC
    LIMIT 10;

Done.


Country,area_water,area_land,water_to_land_ratio
British Indian Ocean Territory,54340,60,905.7
Virgin Islands,1564,346,4.5
Puerto Rico,4921,8870,0.6
"Bahamas, The",3870,10010,0.4
Guinea-Bissau,8005,28120,0.3
Malawi,24404,94080,0.3
Eritrea,16600,101000,0.2
Liberia,15049,96320,0.2
Netherlands,7650,33893,0.2
Uganda,43938,197100,0.2


Again, I'm not sure if the `area_water` includes the territorial water, baseline, internal waters, and exclusive economic zone (EEZ), or anything related to the [maritime boundary](https://en.wikipedia.org/wiki/Maritime_boundary).

<img src="https://upload.wikimedia.org/wikipedia/commons/7/7d/Zonmar-en.svg" width="300" height="400">

### Top 10 countries with most people added to their population in 2016

In [133]:
%%sql
SELECT name AS Country, population, population_growth,
    ROUND(CAST(population AS Float)/100 * population_growth, 0) AS 'added people in 2016'
    FROM facts
    WHERE name != 'World'
    ORDER BY ROUND(CAST(population AS Float)/100 * population_growth, 0) DESC
    LIMIT 10;

Done.


Country,population,population_growth,added people in 2016
India,1251695584,1.22,15270686.0
China,1367485388,0.45,6153684.0
Nigeria,181562056,2.45,4448270.0
Pakistan,199085847,1.46,2906653.0
Ethiopia,99465819,2.89,2874562.0
Bangladesh,168957745,1.6,2703324.0
United States,321368864,0.78,2506677.0
Indonesia,255993674,0.92,2355142.0
"Congo, Democratic Republic of the",79375136,2.45,1944691.0
Philippines,100998376,1.61,1626074.0


I spotted my country there haha.. countries from three continents are listed here, so it seems like the increase in people are pretty balanced around the world.

### Top 10 countries with the highest death rate compared to birth rate

In [134]:
%%sql
SELECT name AS Country, birth_rate, death_rate,
    ROUND(death_rate - birth_rate/death_rate, 2) AS death_birth_ratio
    FROM facts
    ORDER BY death_birth_ratio DESC
    LIMIT 10;

Done.


Country,birth_rate,death_rate,death_birth_ratio
Bulgaria,8.92,14.44,13.82
Ukraine,10.72,14.46,13.72
Latvia,10.0,14.31,13.61
Lithuania,10.1,14.27,13.56
Lesotho,25.47,14.89,13.18
Serbia,9.08,13.66,13.0
Russia,11.6,13.69,12.84
Belarus,10.7,13.36,12.56
Namibia,19.8,13.91,12.49
Hungary,9.16,12.73,12.01


Mostly in Eastern Europe countries.

### Top 10 countries with the highest birth rate compared to the death rate

In [135]:
%%sql
SELECT name AS Country, birth_rate, death_rate,
    ROUND(birth_rate/death_rate, 1)  AS birth_death_ratio
    FROM facts
    ORDER BY birth_death_ratio DESC
    LIMIT 10;

Done.


Country,birth_rate,death_rate,birth_death_ratio
Gaza Strip,31.11,3.04,10.2
Kuwait,19.91,2.18,9.1
Iraq,31.45,3.77,8.3
United Arab Emirates,15.43,1.97,7.8
Oman,24.44,3.36,7.3
Jordan,25.37,3.79,6.7
Solomon Islands,25.77,3.85,6.7
West Bank,22.99,3.5,6.6
Qatar,9.84,1.53,6.4
Marshall Islands,25.6,4.21,6.1


Mostly from Middle Eastern countries.

### Top 10 countries with the highest population/area ratio

In [147]:
%%sql
SELECT name AS Country, population, area,
    ROUND(CAST(population AS Float) / area, 2) AS 'population/area'
    FROM facts
    ORDER BY ROUND(CAST(population AS Float) / area, 2) DESC
    LIMIT 10;

Done.


Country,population,area,population/area
Macau,592731,28,21168.96
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.28
Hong Kong,7141106,1108,6445.04
Gaza Strip,1869055,360,5191.82
Gibraltar,29258,6,4876.33
Bahrain,1346613,760,1771.86
Maldives,393253,298,1319.64
Malta,413965,316,1310.02
Bermuda,70196,54,1299.93


## Conclusion
- China is the country with maximum population, with Pitcairn Islands as the country with minimum population.
- There are several unhabitable countries in the world, such as Antarctica, and other islands and oceans.
- The world's average population is 32,242,667 and average area is 555,093.55. Indonesia's population and area are higher than these.
- The country with the highest growth rate is South Sudan, with Vatican as the country with the lowest growth rate.
- India has added more people into the world in 2016, followed by China and Nigeria. Indonesia is among the top 10.
- High death rates are found in Eastern Europe countries with Bulgaria at the top, while high birth rates are found in Middle Eastern countries with Gaza Strip at the top. Indonesia has higher birth rate than death rate.
- Asia has 3 most crowded countries in the world, which are Macau, Hong Kong and Singapore.
- The `area water` and `area land` data might not be that accurate.